In [ ]:
!pip install --upgrade --force-reinstall \
  transformers==4.40.1 \
  peft==0.10.0 \
  accelerate==0.27.2 \
  datasets==2.18.0 \
  evaluate==0.4.1

In [ ]:
!pip install -U datasets
!pip install huggingface_hub[hf_xet]

In [1]:
import torch
import hf_xet
from transformers import (
    T5ForConditionalGeneration,
    T5Tokenizer,
    pipeline,
    GPT2LMHeadModel,
    GPT2Tokenizer,
    AutoModelForSequenceClassification,
    AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoTokenizer, AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments, Seq2SeqTrainer,
    DataCollatorForSeq2Seq, GenerationConfig
)
from datasets import load_dataset
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from tqdm.auto import tqdm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import gc
import os
import re

Процесс дообучения моделей на датасете Gazeta

In [ ]:
def load_gazeta_dataset(split="train", num_samples=None, min_chars=100, max_chars=5000):

    # Загружает датасет Gazeta с возможностью фильтрации по длине текста

    # :param split: Раздел датасета (train/validation/test)
    # :param num_samples: Ограничение количества примеров
    # :param min_chars: Минимальная длина текста
    # :param max_chars: Максимальная длина текста
    # :return: Отфильтрованный датасет

    print(f"Загружаем Gazeta ({split})...")
    try:
        dataset = load_dataset("IlyaGusev/gazeta", "v1.5", split=split)
    except:
        dataset = load_dataset("IlyaGusev/gazeta", split=split)

    # Фильтрация по длине текста
    filtered = dataset.filter(lambda x: min_chars <= len(x["text"]) <= max_chars)

    if num_samples:
        available_samples = len(filtered)
        actual_samples = min(num_samples, available_samples)
        print(f"Запрошено {num_samples} примеров, доступно {available_samples}. Используем {actual_samples}.")
        return filtered.select(range(actual_samples))
    return filtered

class GazetaSummarizer:
    def __init__(self, model_name="csebuetnlp/mT5_m2o_russian_crossSum"):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(self.device)

        # конфигурация генерации
        self.generation_config = GenerationConfig(
            max_new_tokens=150,
            num_beams=5,
            do_sample=True,
            temperature=0.8,
            top_k=60,
            top_p=0.9,
            repetition_penalty=1.3,
            length_penalty=1.2
        )

    def tokenize_function(self, examples):
        # Функция токенизации с учетом датасета Gazeta
        texts = examples["text"]
        summaries = examples["summary"]

        # Добавляем префикс для T5 моделей
        if 't5' in self.model_name.lower():
            texts = [f"summarize: {text}" for text in texts]

        # Токенизация текстов
        model_inputs = self.tokenizer(
            texts,
            max_length=600,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        # Токенизация пересказов
        with self.tokenizer.as_target_tokenizer():
            labels = self.tokenizer(
                summaries,
                max_length=150,
                truncation=True,
                padding="max_length",
                return_tensors="pt")["input_ids"]

        model_inputs["labels"] = labels
        return model_inputs

    def train(self, train_data, val_data=None, output_dir="./gazeta_summarizer"):

        # Обучение модели

        # Разделение данных если не предоставлен валидационный набор
        if val_data is None:
            split = train_data.train_test_split(test_size=0.15)
            train_data = split["train"]
            val_data = split["test"]

        # Подготовка данных
        train_dataset = train_data.map(
            self.tokenize_function,
            batched=True,
            batch_size=32,
            remove_columns=train_data.column_names
        )
        val_dataset = val_data.map(
            self.tokenize_function,
            batched=True,
            batch_size=32,
            remove_columns=val_data.column_names
        )

        data_collator = DataCollatorForSeq2Seq(
            self.tokenizer,
            model=self.model,
            label_pad_token_id=-100
        )

        # параметры обучения
        training_args = Seq2SeqTrainingArguments(
            output_dir=output_dir,
            evaluation_strategy="steps",
            eval_steps=1000,
            save_strategy="steps",
            save_steps=1000,
            per_device_train_batch_size=4,
            per_device_eval_batch_size=8,
            num_train_epochs=7,  # количество эпох
            learning_rate=3e-5,  # learning rate
            weight_decay=0.01,
            warmup_steps=800,
            logging_dir=f"{output_dir}/logs",
            logging_steps=200,
            fp16=torch.cuda.is_available(),
            load_best_model_at_end=True,
            metric_for_best_model="eval_loss",
            greater_is_better=False,
            predict_with_generate=True,
            report_to="none",
            gradient_accumulation_steps=2,  # Для стабилизации обучения
            lr_scheduler_type="cosine",     # Косинусный планировщик
            save_total_limit=2              # Лучше экономит место
        )

        trainer = Seq2SeqTrainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            tokenizer=self.tokenizer,
            data_collator=data_collator
        )

        print("\n Начинаем обучение...")
        train_result = trainer.train()

        # Сохранение результатов
        trainer.save_model(output_dir)
        trainer.save_state()
        trainer.save_metrics("train", train_result.metrics)

        # Детальная оценка после обучения
        val_metrics = trainer.evaluate(
            metric_key_prefix="final_eval",
            max_length=150,  # Соответствует генерации
            num_beams=5
        )
        trainer.save_metrics("eval", val_metrics)

        print(f"\n Обучение завершено. Модель сохранена в {output_dir}")
        print(f"Train loss: {train_result.metrics['train_loss']:.4f}")
        print(f"Eval loss: {val_metrics['final_eval_loss']:.4f}")

        return train_result.metrics

    def generate_summary(self, text, max_length=150):
        # Генерация пересказа с улучшенной постобработкой
        if 't5' in self.model_name.lower():
            text = f"summarize: {text}"

        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            max_length=600,
            truncation=True,
            padding="max_length"
        ).to(self.device)

        with torch.no_grad():
            output_ids = self.model.generate(
                **inputs,
                generation_config=self.generation_config,
                max_length=max_length
            )

        summary = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)
        return self._postprocess_summary(summary)

    def _postprocess_summary(self, text):
        # Улучшенная постобработка текста

        # Удаление служебных токенов и лишних пробелов
        text = re.sub(r'<extra_id_\d+>|\[.*?\]|(Пересказ|Сводка|Резюме):?\s*', '', text)
        text = re.sub(r'\s+', ' ', text).strip()

        # Коррекция пунктуации
        text = re.sub(r'\s([,.!?])', r'\1', text)
        text = re.sub(r'([,.!?])([А-Яа-я])', r'\1 \2', text)

        # Добавление точки в конце если нужно
        if text and text[-1] not in {'.', '!', '?'}:
            text += '.'

        # Выбор первых 3 предложений (если нужно)
        sentences = re.split(r'(?<=[.!?])\s+', text)
        return ' '.join(sentences[:3]) if len(sentences) > 3 else text

if __name__ == "__main__":
    # 1. Загрузка данных Gazeta
    train_data = load_gazeta_dataset(split="train", num_samples=5000)
    val_data = load_gazeta_dataset(split="validation")

    # 2. Инициализация модели (можно попробовать large если есть ресурсы)
    summarizer = GazetaSummarizer(model_name="csebuetnlp/mT5_m2o_russian_crossSum")

    # 3. Обучение
    summarizer.train(
        train_data=train_data,
        val_data=val_data,
        output_dir="./gazeta_summarizer"
    )

    # 4. Тестирование на примерах
    test_data = load_gazeta_dataset(split="test", num_samples=5)
    for i, example in enumerate(test_data):
        print("\n" + "="*50)
        print(f"📰 Пример {i+1}")
        print("Оригинал:", example["text"][:200].replace("\n", " ") + "...")
        print("Эталон:", example["summary"])
        print("Сгенерировано:", summarizer.generate_summary(example["text"]))

Тест на произвольном тексте

In [ ]:
model_path = "./gazeta_summarizer"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

def summarize(text):
    inputs = tokenizer(
        f"summarize: {text}",
        return_tensors="pt",
        max_length=600,
        truncation=True,
        padding=True
    )

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=150,
            num_beams=5,
            do_sample=True,
            top_k=60,
            top_p=0.9,
            temperature=0.8,
            repetition_penalty=1.3,
            length_penalty=1.2
        )

    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [ ]:
texts='Почти 1,3 тысячи жителей курского приграничья, которые числились в реестре утративших связь с родственниками из-за действий ВСУ, найдены на данный момент. Об этом сообщил врио губернатора региона Александр Хинштейн. Кроме того, по его словам, установлено примерное нахождение еще 421 человека. И цифра людей, чье местонахождение сегодня, увы, нам неизвестно, - на данную минуту это 576 человек, - добавил Хинштейн, отметив, что это очень большая цифра, но она корректная, поскольку сложилась из ряда источников. С момента создания реестра в него было внесено 2287 человек, уточнил врио губернатора региона. Он также пояснил, что при формировании соответствующего реестра собирались данные от целого ряда ведомств. Власти в том числе запрашивали в СФР данные о пенсионерах, которые с августа 2024 года не получали пенсию из приграничных районов.'

In [ ]:
print(summarize(texts))

zip модели, для последующей скачки

In [ ]:
!zip -r gazeta_summarizer.zip ./gazeta_summarizer